#  A - Data Cleaning

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
traindf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')

In [3]:
dfs = [traindf, testdf]
for df in dfs:
    df.replace({'Gender' : 'Male', 'Married' : 'Yes', 'Education' : 'Graduate', 'Self_Employed' : 'Yes', 'Property_Area' : 'Urban'}, 1, inplace = True) 
    df.replace({'Gender' : 'Female', 'Married' : 'No', 'Education' : 'Not Graduate', 'Self_Employed' : 'No', 'Property_Area' : 'Rural'}, 0, inplace = True)
    df.replace({'Property_Area' : 'Semiurban', 'Dependents' : '3+'}, 3, inplace = True)
traindf.replace(np.nan, 0, inplace = True)
testdf.replace(np.nan, 0, inplace = True)

In [4]:
XTrain = traindf.drop(['Loan_Status'], axis = 1)
YTrain = pd.DataFrame(traindf['Loan_Status'])
XTest = testdf.drop(['Loan_ID'], axis = 1)
YTrain = YTrain['Loan_Status'].tolist()
XTrain.drop(['Loan_ID'], axis = 1, inplace = True)
YTest = YTrain[:367]

========================================================================================================================

# B  - One Hot Encoding

In [5]:
XTrain = XTrain.astype('float32')
XTest = XTest.astype('float32')

In [6]:
encoder = OneHotEncoder(categories = 'auto')
trainLabels = encoder.fit_transform(XTrain).toarray()
testLabels = encoder.fit_transform(XTest).toarray()

# C - Prediction using k Nearest Neigbors Model

In [7]:
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(trainLabels, YTrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [8]:
accuracy_score(YTest, neigh.predict(testLabels))

ValueError: query data dimension must match training data dimension

In [9]:
pd.DataFrame(neigh.predict(testLabels)).to_csv('kNearestNeighbors.csv')

ValueError: query data dimension must match training data dimension

========================================================================================================================

# D - Prediction using Logistic Regression Model

In [10]:
min_max = MinMaxScaler()
XTrain = min_max.fit_transform(XTrain[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])
XTest = min_max.fit_transform(XTest[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])

In [11]:
log = LogisticRegression(penalty = 'l2', C = 1, solver = 'lbfgs')
log.fit(XTrain, YTrain)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
accuracy_score(YTest, log.predict(XTest))

0.6457765667574932

In [13]:
pd.DataFrame(log.predict(XTest)).to_csv('LosticRegression.csv')